- Review the relevant models: bag-of-words, Word2Vec, Doc2Vec

- Load and preprocess the training and test corpora (see Corpus)

- Train a Doc2Vec Model model using the training corpus

- Demonstrate how the trained model can be used to infer a Vector

- Assess the model

- Test the model on the test corpus

In [1]:
import logging
import pprint
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### practise bag-of-words with Gensim

In [2]:
text_corpus = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

In [3]:
# Create a set of frequent words
stoplist = set('for a of the and to in'.split(' '))
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in text_corpus]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
pprint.pprint(processed_corpus)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


Before proceeding, we want to associate each word in the corpus with a unique integer ID. We can do this using the gensim.corpora.Dictionary class. This dictionary defines the vocabulary of all words that our processing knows about.

In [4]:
from gensim import corpora
dictionary = corpora.Dictionary(processed_corpus)

2020-03-20 10:01:58,278 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-03-20 10:01:58,279 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [5]:
dictionary.token2id

{'computer': 0,
 'human': 1,
 'interface': 2,
 'response': 3,
 'survey': 4,
 'system': 5,
 'time': 6,
 'user': 7,
 'eps': 8,
 'trees': 9,
 'graph': 10,
 'minors': 11}

suppose we wanted to vectorize the phrase “Human computer interaction” (note that this phrase was not in our original corpus). We can create the bag-of-word representation for a document using the doc2bow method of the dictionary, which returns a sparse representation of the word counts:

In [6]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
pprint.pprint(new_vec)

[(0, 1), (1, 1)]


In [7]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
pprint.pprint(bow_corpus)

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]


In [8]:
from gensim import models

# train the model
tfidf = models.TfidfModel(bow_corpus)

# transform the "system minors" string
words = "system minors".lower().split()
print(tfidf[dictionary.doc2bow(words)])

2020-03-20 10:02:05,883 : INFO : collecting document frequencies
2020-03-20 10:02:05,884 : INFO : PROGRESS: processing document #0
2020-03-20 10:02:05,885 : INFO : calculating IDF weights for 9 documents and 12 features (28 matrix non-zeros)


[(5, 0.5898341626740045), (11, 0.8075244024440723)]


In [9]:
from gensim import similarities

index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=12)

2020-03-20 10:02:09,752 : INFO : creating sparse index
2020-03-20 10:02:09,753 : INFO : creating sparse matrix from corpus
2020-03-20 10:02:09,753 : INFO : PROGRESS: at document #0
2020-03-20 10:02:09,754 : INFO : created <9x12 sparse matrix of type '<class 'numpy.float32'>'
	with 28 stored elements in Compressed Sparse Row format>


In [10]:
query_document = 'system engineering'.split()
query_bow = dictionary.doc2bow(query_document)
sims = index[tfidf[query_bow]]
print(list(enumerate(sims)))

[(0, 0.0), (1, 0.32448703), (2, 0.41707572), (3, 0.7184812), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)]


How to read this output? Document 3 has a similarity score of 0.718=72%, document 2 has a similarity score of 42% etc. We can make this slightly more readable by sorting:

In [12]:
for document_number, score in sorted(enumerate(sims), key = lambda x: x[1], reverse=True):
    print(document_number, score)

3 0.7184812
2 0.41707572
1 0.32448703
0 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0


# Prepare the Training and Test Data

In [14]:
import os
import gensim
# Set file names for train and test data
test_data_dir = os.path.join(gensim.__path__[0], 'test', 'test_data')
lee_train_file = os.path.join(test_data_dir, 'lee_background.cor')
lee_test_file = os.path.join(test_data_dir, 'lee.cor')

In [16]:
import smart_open

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

In [19]:
train_corpus[:2]

[TaggedDocument(words=['hundreds', 'of', 'people', 'have', 'been', 'forced', 'to', 'vacate', 'their', 'homes', 'in', 'the', 'southern', 'highlands', 'of', 'new', 'south', 'wales', 'as', 'strong', 'winds', 'today', 'pushed', 'huge', 'bushfire', 'towards', 'the', 'town', 'of', 'hill', 'top', 'new', 'blaze', 'near', 'goulburn', 'south', 'west', 'of', 'sydney', 'has', 'forced', 'the', 'closure', 'of', 'the', 'hume', 'highway', 'at', 'about', 'pm', 'aedt', 'marked', 'deterioration', 'in', 'the', 'weather', 'as', 'storm', 'cell', 'moved', 'east', 'across', 'the', 'blue', 'mountains', 'forced', 'authorities', 'to', 'make', 'decision', 'to', 'evacuate', 'people', 'from', 'homes', 'in', 'outlying', 'streets', 'at', 'hill', 'top', 'in', 'the', 'new', 'south', 'wales', 'southern', 'highlands', 'an', 'estimated', 'residents', 'have', 'left', 'their', 'homes', 'for', 'nearby', 'mittagong', 'the', 'new', 'south', 'wales', 'rural', 'fire', 'service', 'says', 'the', 'weather', 'conditions', 'which', '

### Training the model

Now, we’ll instantiate a Doc2Vec model with a vector size with 50 dimensions and iterating over the training corpus 40 times. We set the minimum word count to 2 in order to discard words with very few occurrences. (Without a variety of representative examples, retraining such infrequent words can often make a model worse!) Typical iteration counts in the published Paragraph Vector paper results, using 10s-of-thousands to millions of docs, are 10-20. More iterations take more time and eventually reach a point of diminishing returns.

However, this is a very very small dataset (300 documents) with shortish documents (a few hundred words). Adding training passes can sometimes help with such small datasets.

In [21]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(train_corpus)

2020-03-20 10:22:50,222 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2020-03-20 10:22:50,223 : INFO : collecting all words and their counts
2020-03-20 10:22:50,223 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-03-20 10:22:50,233 : INFO : collected 6981 word types and 300 unique tags from a corpus of 300 examples and 58152 words
2020-03-20 10:22:50,233 : INFO : Loading a fresh vocabulary
2020-03-20 10:22:50,243 : INFO : effective_min_count=2 retains 3955 unique words (56% of original 6981, drops 3026)
2020-03-20 10:22:50,244 : INFO : effective_min_count=2 leaves 55126 word corpus (94% of original 58152, drops 3026)
2020-03-20 10:22:50,250 : INFO : deleting the raw counts dictionary of 6981 items
2020-03-20 10:22:50,250 : INFO : sample=0.001 downsamples 46 most-common words
2020-03-20 10:22:50,251 : INFO : downsampling leaves estimated 42390 word corpus (76.9% of prior 55126)
2020-03-20 10:22:50,256 : INFO : est

In [22]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

2020-03-20 10:23:11,470 : INFO : training model with 3 workers on 3955 vocabulary and 50 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2020-03-20 10:23:11,547 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-03-20 10:23:11,558 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-03-20 10:23:11,559 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-03-20 10:23:11,559 : INFO : EPOCH - 1 : training on 58152 raw words (42642 effective words) took 0.1s, 490133 effective words/s
2020-03-20 10:23:11,586 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-03-20 10:23:11,588 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-03-20 10:23:11,589 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-03-20 10:23:11,589 : INFO : EPOCH - 2 : training on 58152 raw words (42791 effective words) took 0.0s, 1513507 effective words/s
2020-03-20 10:23:11,618 : INFO : worker

In [29]:
model.corpus_count

300

In [28]:
model.wv.vocab['penalty'].count

4

In [23]:
vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
print(vector)

[ 0.06732272 -0.12476047  0.11916967 -0.02924351 -0.3081726   0.2028
  0.00062394  0.10252877 -0.19012024 -0.09102704 -0.02804117  0.04043559
  0.0432324  -0.03685858 -0.11482809  0.2632684   0.07519898 -0.16111656
  0.30788338 -0.00605491  0.11093695 -0.03716284 -0.15965474 -0.11657619
  0.19527799 -0.10133257  0.02801606 -0.1710302   0.05739621 -0.04919119
  0.02137833 -0.0624913   0.01614645  0.255878    0.1485272   0.08676764
  0.0472293   0.09611478  0.0510371   0.07610098  0.09373116 -0.09922076
  0.00806367 -0.01895287  0.20629326  0.12039571 -0.21268377 -0.06012771
 -0.01002343 -0.04122327]


### Assessing the Model

To assess our new model, we’ll first infer new vectors for each document of the training corpus, compare the inferred vectors with the training corpus, and then returning the rank of the document based on self-similarity. Basically, we’re pretending as if the training corpus is some new unseen data and then seeing how they compare with the trained model. The expectation is that we’ve likely overfit our model (i.e., all of the ranks will be less than 2) and so we should be able to find similar documents very easily. Additionally, we’ll keep track of the second ranks for a comparison of less similar documents.

In [30]:
ranks = []
second_ranks = []

for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn = len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

2020-03-20 10:36:35,939 : INFO : precomputing L2-norms of doc weight vectors


In [31]:
import collections

counter = collections.Counter(ranks)
print(counter)

Counter({0: 291, 1: 9})


Basically, greater than 95% of the inferred documents are found to be most similar to itself and about 5% of the time it is mistakenly most similar to another document. Checking the inferred-vector against a training-vector is a sort of ‘sanity check’ as to whether the model is behaving in a usefully consistent manner, though not a real ‘accuracy’ value.

This is great and not entirely surprising. We can take a look at an example:

In [34]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)

for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (299): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with his form it not v

In [36]:
# Pick a random document from the corpus and infer a vector from the model
import random
doc_id = random.randint(0, len(train_corpus) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Train Document (260): «traveland wholly owned travel centres have ceased operating from today leaving more than staff seeking other jobs the failed company administrators say they have buyer for traveland franchise network but have not been able to save the company stores one of the administrators richard albarran says the deal which is yet to be approved by committee formed today of creditors will unfortunately leave hundreds of people who have booked holidays through the company stores out of pocket the dollar value is approximately tad over million mr albarran said they will now be entitled to make claims through the travel compensation fund the meeting of company creditors was told this morning staff are owed nearly million in entitlements the australian services union luke foley says he will be doing everything he can to ensure they receive every cent ansett administrators are liable for perhaps the lion share of those employee entitlements we re confident that ll be met mr foley 

### Testing the Model

Using the same approach above, we’ll infer the vector for a randomly chosen test document, and compare the document to our model by eye.

In [37]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (24): «nigerian president olusegun obasanjo said he will weep if single mother sentenced to death by stoning for having child out of wedlock is killed but added he has faith the court system will overturn her sentence obasanjo comments late saturday appeared to confirm he would not intervene directly in the case despite an international outcry»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST (23, 0.7190220355987549): «americans fears about airplane security continue to increase after man made it through two separate flights with loaded gun in his carry on luggage the man was finally stopped before boarding third plane in memphis the man had travelled from florida to atlanta and then atlanta to memphis he was attempting to board his return flight last night when he was stopped by security personnel for random check they discovered loaded mm beretta semi automatic pistol in his hand luggage the man acknowledged the gun was his and was release